In [2]:
import pandas as pd
date='01'
df=pd.read_csv(f'./Latest_ETM_Data/Final_Ticket_data_2024-07-{date}.csv')
df.shape

(1978353, 32)

In [3]:
max_value = df['source_zonal_centroid_number'].max()
print(f"The maximum value in source_zonal_centroid_number is: {max_value}")

max_dest = df['destination_zonal_centroid_number'].max()
print(f"The maximum value in dstination_zonal_centroid_number is: {max_dest}")
#3039

The maximum value in source_zonal_centroid_number is: 3039.0
The maximum value in dstination_zonal_centroid_number is: 3039.0


In [4]:
all_zonal_centroids = range(1, 3039)

# Create all possible source-destination pairs
all_pairs = pd.MultiIndex.from_product(
    [all_zonal_centroids, all_zonal_centroids],
    names=['source_zonal_centroid_number', 'destination_zonal_centroid_number']
)

# Group and sum passenger counts, then reindex to include all pairs
grouped = df.groupby(['source_zonal_centroid_number', 'destination_zonal_centroid_number'])['passenger_count'].sum()
od_matrix = grouped.reindex(all_pairs, fill_value=0).unstack(fill_value=0)

# Add a column for the row sums
od_matrix['Total Passengers'] = od_matrix.sum(axis=1)

od_matrix.to_excel('ODM_01.xlsx')

In [45]:
# Filter rows where 'source_stop_no' is NaN
ticket_df_na_source_stop_no = df[df['source_stop_no'].isna()]
ticket_df_na_source_stop_no_unique = ticket_df_na_source_stop_no[['source','Direction_route']].drop_duplicates()

ticket_df_na_source_stop_no_unique

,source,Direction_route
0,ZERO VALUE TICKET,UP
135,ZERO VALUE TICKET,DOWN
2524,EGMORE,DOWN
2567,TONDIARPET,DOWN
8258,THIRUVANMIYUR OR MARUDEESWARAR TEMPLE,DOWN
...,...,...
1657981,VEERABADRA NAGAR,NaN
1696522,MANDAVELI,NaN
1722051,PAMMAL,NaN
1753701,CHIDAMBARAM STORE,NaN


In [46]:
# Filter rows where 'destination_stop_no' is NaN
ticket_df_na_destination_stop_no = df[df['destination_stop_no'].isna()]
ticket_df_na_destination_stop_no_unique = ticket_df_na_destination_stop_no[['destination','Direction_route']].drop_duplicates()

ticket_df_na_destination_stop_no_unique

,destination,Direction_route
0,ZERO VALUE TICKET,UP
135,ZERO VALUE TICKET,DOWN
2541,TONDIARPET,DOWN
27085,ZERO VALUE TICKET,NaN
28959,THIRUVANMIYUR OR MARUDEESWARAR TEMPLE,DOWN
...,...,...
1748902,MADHAVARAM,NaN
1753701,NANGANALLUR BUS TERMINUS,NaN
1778085,LAKSHMIPURAM,NaN
1790069,PUZHUTHIVAKKAM,DOWN


In [47]:
# Combine the unique 'source' and 'destination' stops where stop_no is NaN
combined_unique_stops = pd.concat([
    ticket_df_na_source_stop_no_unique.rename(columns={'source': 'stop'}),  # Rename source to stop
    ticket_df_na_destination_stop_no_unique.rename(columns={'destination': 'stop'})  # Rename destination to stop
], ignore_index=True).drop_duplicates()

# Print the final unique stops
combined_unique_stops.to_csv('remaining_values.csv')

In [48]:
combined_unique_stops

,stop,Direction_route
0,ZERO VALUE TICKET,UP
1,ZERO VALUE TICKET,DOWN
2,EGMORE,DOWN
3,TONDIARPET,DOWN
4,THIRUVANMIYUR OR MARUDEESWARAR TEMPLE,DOWN
...,...,...
395,MADUVANKARAI,NaN
396,POZHICHALUR,NaN
399,NANGANALLUR BUS TERMINUS,NaN
400,LAKSHMIPURAM,NaN


In [49]:
cols=['Trip_Origin','Trip_Destination','Direction_route']
df=df[cols]
df

,Trip_Origin,Trip_Destination,Direction_route
0,ENNORE,M.G.R.KOYAMBEDU,UP
1,ENNORE,M.G.R.KOYAMBEDU,UP
2,ENNORE,M.G.R.KOYAMBEDU,UP
3,ENNORE,M.G.R.KOYAMBEDU,UP
4,ENNORE,M.G.R.KOYAMBEDU,UP
...,...,...,...
1927193,TAMBARAM WEST,SAIDAPET,DOWN
1927194,TAMBARAM WEST,SAIDAPET,DOWN
1927195,TAMBARAM WEST,SAIDAPET,DOWN
1927196,TAMBARAM WEST,SAIDAPET,DOWN


In [50]:
# First operation: Filter rows where 'Direction_route' is NaN and remove duplicates
result = df[df['Direction_route'].isna()].drop_duplicates()
result.to_csv(f'UP_DOWN_{date}.csv')

In [51]:
result

,Trip_Origin,Trip_Destination,Direction_route
27085,NALLUR,KUNDRATHUR B.S,NaN
84527,SEMMANCHERI S.C.B,ADYAR (DEPOT),NaN
93063,THIRUVANMIYUR,OKK.THORAIPAKM SEC.COLONY,NaN
93330,ADYAR B.S,PERUMPAKKAM S.C.B,NaN
93409,PERUMPAKKAM S.C.B,BESANT NAGAR,NaN
...,...,...,...
1753699,KILAMBAKKAM B.S,NANGANALLUR,NaN
1778085,POONAMALLEE B.S,VADAKKU MALAIAMBAKKAM,NaN
1830023,PERIYAR NAGAR,SRINIVASA NAGAR,NaN
1830564,SRINIVASA NAGAR,OTTERI,NaN
